### 11.3 Finding frequent itemsets with the Apriori algorithm

In [23]:
def loadDataSet():
    '''
    生成一个简单的数据集
    :return: 数据集
    '''
    return [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

In [22]:
def createC1(dataSet):
    '''
    创建初始的单项集
    :param dataSet: 数据集
    :return: 单项集
    '''
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if [item] not in C1:
                C1.append([item])
    C1.sort()
    return map(frozenset, C1)  # frozenset是不可变集合

In [56]:
def scanD(D, Ck, minSupport):
    """scanD（计算候选数据集 CK 在数据集 D 中的支持度，并返回支持度大于最小支持度 minSupport 的数据）

    Args:
        D 数据集
        Ck 候选项集列表
        minSupport 最小支持度
    Returns:
        retList 支持度大于 minSupport 的集合
        supportData 候选项集支持度数据
    """
    D = list(D)
    Ck = list(Ck)
    # print("D:", D)
    # print("Ck:", Ck)
    # ssCnt 临时存放选数据集 Ck 的频率. 例如: a->10, b->5, c->8    
    ssCnt = {}
    for tid in D:
        for can in Ck:
            # s.issubset(t)  测试是否 s 中的每一个元素都在 t 中
            if can.issubset(tid):
                if can not in ssCnt:
                    ssCnt[can] = 1
                else:
                    ssCnt[can] += 1
    numItems = float(len(D)) # 数据集 D 的数量
    retList = []
    supportData = {}
    for key in ssCnt:
        # 支持度 = 候选项（key）出现的次数 / 所有数据集的数量
        support = ssCnt[key]/numItems
        if support >= minSupport:
            # 在 retList 的首位插入元素，只存储支持度满足频繁项集的值
            retList.insert(0, key)
        # 存储所有的候选项（key）和对应的支持度（support）
        supportData[key] = support
    return retList, supportData

In [50]:
dataSet = loadDataSet() # 加载数据集
print("dataSet:", dataSet)

dataSet: [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]


In [62]:
C1 = createC1(dataSet) # 创建初始单项集 
D =map(set, dataSet) # 将数据集转换为集合
L1, supportData0 = scanD(D, C1, 0.5) # 计算频繁项集和支持度

In [63]:
def aprioriGen(Lk, k):
    '''
    生成候选项集
    :param Lk: 频繁项集
    :param k: 项集的大小
    :return: 候选项集
    '''
    retList = [] # 存储候选项集
    lenLk = len(Lk) # 频繁项集的长度
    for i in range(lenLk): # 遍历频繁项集
        for j in range(i + 1, lenLk): # 遍历频繁项集
            L1 = list(Lk[i])[:k - 1] # 前 k-1 个元素
            L2 = list(Lk[j])[:k - 1] # 前 k-1 个元素
            L1.sort()
            L2.sort()
            if L1 == L2:
                retList.append(Lk[i] | Lk[j]) # 集合的并集
    return retList

In [64]:
def apriori(dataSet, minSupport=0.5):
    '''
    apriori 算法
    :param dataSet: 数据集
    :param minSupport: 最小支持度
    :return: 频繁项集和支持度
    '''
    C1 = createC1(dataSet) # 创建初始单项集
    D = list(map(set, dataSet)) # 将数据集转换为集合
    L1, supportData = scanD(D, C1, minSupport) # 计算频繁项集和支持度
    L = [L1] # 存储频繁项集
    k = 2 # 项集的大小
    while (len(L[k - 2]) > 0): # 当频繁项集不为空时
        Ck = aprioriGen(L[k - 2], k) # 生成候选项集
        Lk, supK = scanD(D, Ck, minSupport) # 计算频繁项集和支持度
        supportData.update(supK) # 更新支持度数据
        L.append(Lk) # 存储频繁项集
        k += 1 # 项集的大小加 1
    return L, supportData

In [66]:
L, supportData = apriori(dataSet, minSupport=0.5) # 计算频繁项集和支持度
# print("L:", L) # 打印频繁项集

In [67]:
def calcConf(freqSet, H, supportData, minConf=0.7):
    '''
    计算置信度
    :param freqSet: 频繁项集
    :param H: 候选项集
    :param supportData: 支持度数据
    :param minConf: 最小置信度
    :return: 置信度大于最小置信度的规则
    '''
    prunedH = [] # 存储置信度大于最小置信度的规则
    for conseq in H: # 遍历候选项集
        conf = supportData[freqSet] / supportData[freqSet - conseq] # 计算置信度
        if conf >= minConf: # 如果置信度大于最小置信度
            print(freqSet - conseq, '-->', conseq, 'conf:', conf) # 打印规则和置信度
            prunedH.append(conseq) # 存储置信度大于最小置信度的规则
    return prunedH

In [69]:
def rulesFromConseq(freqSet, H, supportData, minConf=0.7):
    '''
    递归计算频繁项集的规则
    :param freqSet: 频繁项集
    :param H: 候选项集
    :param supportData: 支持度数据
    :param minConf: 最小置信度
    :return: 置信度大于最小置信度的规则
    '''
    m = len(H[0]) # 候选项集的大小
    if len(freqSet) > (m + 1): # 如果频繁项集的大小大于候选项集的大小加 1
        Hmp1 = aprioriGen(H, m + 1) # 生成候选项集
        Hmp1 = calcConf(freqSet, Hmp1, supportData, minConf) # 计算置信度
        if len(Hmp1) > 1: # 如果候选项集不为空
            rulesFromConseq(freqSet, Hmp1, supportData, minConf) # 递归调用函数

In [70]:
def generateRules(L, supportData, minConf=0.7):
    '''
    生成关联规则
    :param L: 频繁项集
    :param supportData: 支持度数据
    :param minConf: 最小置信度
    :return: 置信度大于最小置信度的规则
    '''
    bigRuleList = [] # 存储置信度大于最小置信度的规则
    for i in range(1, len(L)): # 遍历频繁项集
        for freqSet in L[i]: # 遍历频繁项集
            H1 = [frozenset([item]) for item in freqSet] # 创建候选项集
            if (i > 1): # 如果频繁项集的大小大于 1
                rulesFromConseq(freqSet, H1, supportData, minConf) # 生成规则
            else: # 如果频繁项集的大小等于 1
                calcConf(freqSet, H1, supportData, minConf) # 计算置信度
                
    return bigRuleList # 返回置信度大于最小置信度的规则